# Prepare Finetuning Data

In [15]:
import pickle
from Grading_Functions import *
from Evaluation_Functions import *
import pandas as pd
from unidecode import unidecode

# Load conversations
with open('Conversation_Lists.pkl', 'rb') as file:
    loaded_data = pickle.load(file)


In [16]:
mtm_list = [mtm for mtm in loaded_data.keys()]
conversations_list = [conversation for conversation in loaded_data.values()]

print(len(mtm_list))
print(len(conversations_list))


32
32


In [17]:
# Iterate over elements of dictionary loaded_data
# Load the list at that element
# Each element in that list is a list (conversation)
# Save fill lists to store model_task_method and the conversation number
model_task_methods = []
conversation_numbers = []
flat_conversations = []
for model_task_method, conversations in zip(mtm_list, conversations_list):
    for i, conversation in enumerate(conversations):
        model_task_methods.append(model_task_method)
        conversation_numbers.append(i+1)
        # Convert all conversation elements to strings
        all_string_conversation = [str(element) for element in conversation]
        flat_conversations.append(all_string_conversation)


In [18]:
# Create cw_conversations_list
cw_model_task_methods = []
cw_flat_conversations_no_metadata = []
cw_conversation_numbers = []
cw_sentences = []
for index in range(len(flat_conversations)):
    if 'cw' in model_task_methods[index]:
        if 'gpt4' in model_task_methods[index]:
            cw_model_task_methods.append(model_task_methods[index])
            cw_flat_conversations_no_metadata.append(gpt4_cw_cleanup(flat_conversations[index]))
            cw_conversation_numbers.append(conversation_numbers[index])
        else:
            cw_model_task_methods.append(model_task_methods[index])
            cw_flat_conversations_no_metadata.append(td3_cw_cleanup(flat_conversations[index]))
            cw_conversation_numbers.append(conversation_numbers[index])


In [19]:
# Create list of responses to grade
responses = []
for index in range(len(cw_flat_conversations_no_metadata)):
    # Get appropriate message
    if 'self_refine' in cw_model_task_methods[index] or 'tree_of_thought' in cw_model_task_methods[index]:
        responses.append(cw_flat_conversations_no_metadata[index][-3])
    else:
        responses.append(cw_flat_conversations_no_metadata[index][-1])


In [20]:
# Put cw_model_task_methods, cw_conversation_numbers, and responses into a dataframe
responses_df = pd.DataFrame({'model_task_method': cw_model_task_methods,
                                'conversation_number': cw_conversation_numbers,
                                'response': responses})

responses_df


,model_task_method,conversation_number,response
0,td3_cw_direct_prompting_responses,1,Learning to do a handstand is a fun activity f...
1,td3_cw_direct_prompting_responses,2,The hawk was used to hunting what he needed fo...
2,td3_cw_direct_prompting_responses,3,I love the smell of roasting almonds in the ki...
3,td3_cw_direct_prompting_responses,4,Ralph's bedroom was routinely filled up with s...
4,td3_cw_direct_prompting_responses,5,Joe had a unique way of dealing with the hospi...
...,...,...,...
1595,gpt4_cw_manual_cot_responses,96,"For as long as I can remember, my brother had ..."
1596,gpt4_cw_manual_cot_responses,97,Ideas:\n1. Make the passage about learning les...
1597,gpt4_cw_manual_cot_responses,98,Shawn was lagging behind in his political scie...
1598,gpt4_cw_manual_cot_responses,99,Ideas:\n1. Make the passage about a child play...


In [21]:
# Load in cw_graded.xlsx
cw_graded = pd.read_excel('cw_graded.xlsx')

cw_graded


,model_task_method,conversation_number,coherence_1_incoherent_10_very_coherent,task_constraints_followed_0_not_followed_1_followed,ease_of_review_1_easy_10_hard
0,td3_cw_direct_prompting_responses,1,1,1.0,NaN
1,td3_cw_direct_prompting_responses,2,7,0.0,NaN
2,td3_cw_direct_prompting_responses,3,1,1.0,NaN
3,td3_cw_direct_prompting_responses,4,10,1.0,NaN
4,td3_cw_direct_prompting_responses,5,4,1.0,NaN
...,...,...,...,...,...
1595,gpt4_cw_manual_cot_responses,96,10,NaN,NaN
1596,gpt4_cw_manual_cot_responses,97,7,NaN,NaN
1597,gpt4_cw_manual_cot_responses,98,7,NaN,NaN
1598,gpt4_cw_manual_cot_responses,99,7,NaN,NaN


In [22]:
# Create full_human_data by merging responses_df and cw_graded
full_human_data = pd.merge(responses_df, cw_graded, how='left', on=['model_task_method', 'conversation_number'])

# Keep only model_task_method, conversation_number, response, and coherence_1_incoherent_10_very_coherent
full_human_data = full_human_data[['model_task_method', 'conversation_number', 'response', 'coherence_1_incoherent_10_very_coherent']]

# Save to csv
full_human_data.to_csv('full_human_data.csv', index=False)

full_human_data


,model_task_method,conversation_number,response,coherence_1_incoherent_10_very_coherent
0,td3_cw_direct_prompting_responses,1,Learning to do a handstand is a fun activity f...,1
1,td3_cw_direct_prompting_responses,2,The hawk was used to hunting what he needed fo...,7
2,td3_cw_direct_prompting_responses,3,I love the smell of roasting almonds in the ki...,1
3,td3_cw_direct_prompting_responses,4,Ralph's bedroom was routinely filled up with s...,10
4,td3_cw_direct_prompting_responses,5,Joe had a unique way of dealing with the hospi...,4
...,...,...,...,...
1595,gpt4_cw_manual_cot_responses,96,"For as long as I can remember, my brother had ...",10
1596,gpt4_cw_manual_cot_responses,97,Ideas:\n1. Make the passage about learning les...,7
1597,gpt4_cw_manual_cot_responses,98,Shawn was lagging behind in his political scie...,7
1598,gpt4_cw_manual_cot_responses,99,Ideas:\n1. Make the passage about a child play...,7
